In [81]:
import tensorflow as tf
import numpy as np

In [82]:

hidden_units = 10
vocab_size = 10000
embedding_size = 4
layers_num = 10
batch_size = 5
num_steps = 3
tf.reset_default_graph()

In [83]:
source_inputs = tf.placeholder(shape=(batch_size, num_steps), dtype=tf.int32, name='source_inputs')
target_inputs = tf.placeholder(shape=(batch_size, num_steps), dtype=tf.int32, name='y_inputs')

In [84]:
source_inputs

<tf.Tensor 'source_inputs:0' shape=(5, 3) dtype=int32>

In [85]:
target_inputs

<tf.Tensor 'y_inputs:0' shape=(5, 3) dtype=int32>

In [86]:
embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), dtype=tf.float32)

In [87]:
inputs_embedded = tf.nn.embedding_lookup(embedding, source_inputs)
inputs_embedded

<tf.Tensor 'embedding_lookup:0' shape=(5, 3, 4) dtype=float32>

In [88]:
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_units)
    return cell

In [89]:
cell_fw = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(layers_num)], state_is_tuple=True)
cell_bw = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(layers_num)], state_is_tuple=True)

initial_state_fw = cell_fw.zero_state(batch_size, tf.float32)
initial_state_bw = cell_bw.zero_state(batch_size, tf.float32)

In [90]:
((outputs_fw,
  outputs_bw),
 (final_state_fw,
  final_state_bw)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw,
                                                      cell_bw=cell_bw,
                                                      inputs=inputs_embedded,
                                                      initial_state_fw=initial_state_fw,
                                                      initial_state_bw=initial_state_bw,
                                                      dtype=tf.float32,
                                                      time_major=False))

In [91]:
encoder_outputs = tf.concat((outputs_fw, outputs_bw), 2)
encoder_outputs

<tf.Tensor 'concat:0' shape=(5, 3, 20) dtype=float32>

In [95]:
output = tf.reshape(encoder_outputs, [-1, hidden_units*2])
output

<tf.Tensor 'Reshape_1:0' shape=(15, 20) dtype=float32>

In [96]:
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [100]:
num_classes = 4
with tf.variable_scope('outputs'):
    softmax_w = weight_variable([hidden_units * 2, num_classes])
    softmax_b = bias_variable([num_classes])
    y_pred = tf.matmul(output, softmax_w) + softmax_b
    y_pred

In [101]:
y_pred

<tf.Tensor 'outputs_4/add:0' shape=(15, 4) dtype=float32>